In [8]:
import os
import numpy
import pymysql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

loading in the data frames

In [9]:
#loading in the nfl team data frame
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file = os.path.join(data_dir, 'nfl_teams.csv')
nfl_teams = pd.read_csv(data_file)

#loading in the nfl stadiums data frame
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file2 = os.path.join(data_dir, 'nfl_stadiums.csv')
nfl_stadiums = pd.read_csv(data_file2,encoding='ISO-8859-1')

#loading in the spread of games and scores
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file3 = os.path.join(data_dir, 'spreadspoke_scores.csv')
spread = pd.read_csv(data_file3, header=0)

cleaning the files

In [10]:
#Removing weather because we are bringing our own weather in using an api
spread = spread[['schedule_date',"schedule_season","schedule_week","schedule_playoff","team_home","score_home","score_away","team_away","team_favorite_id","spread_favorite","over_under_line","stadium"]]
desired_years = [2022,2023]
spread = spread[spread['schedule_season'].isin(desired_years)]
#for the home team column
spread['team_home'] = spread['team_home'].replace('San Diego Chargers', 'Los Angeles Chargers')
spread['team_home'] = spread['team_home'].replace('Washington Football Team', 'Washington Commanders')
spread['team_home'] = spread['team_home'].replace('Washington Redskins', 'Washington Commanders')
#for the away team column
spread['team_away'] = spread['team_away'].replace('San Diego Chargers', 'Los Angeles Chargers')
spread['team_away'] = spread['team_away'].replace('Washington Football Team', 'Washington Commanders')
spread['team_away'] = spread['team_away'].replace('Washington Redskins', 'Washington Commanders')
#renaming to rematch the cols in the sql dataframe
spread = spread.rename(columns={"schedule_season":"season","schedule_week":"scheduled week","schedule_playoff":"playoff","team_home":"home team","score_home":"home score","score_away":"away score","team_away":"away team","team_favorite_id":"favorite team","spread_favorite":"spread favor","over_under_line":"over under","stadium":"stadium_name"})
spread["id"] = range(1,len(spread)+1)
print(spread.head())
#cleaning the nfl teams data frame
nfl_teams = nfl_teams[["team_name","team_name_short","team_id","team_conference","team_division"]]
nfl_teams = nfl_teams.dropna(subset=['team_division'])
nfl_teams = nfl_teams.drop(nfl_teams[nfl_teams['team_name'] == 'San Diego Chargers'].index)
nfl_teams = nfl_teams.drop(nfl_teams[nfl_teams['team_name'] == 'Washington Football Team'].index)
nfl_teams = nfl_teams.drop(nfl_teams[nfl_teams['team_name'] == 'Washington Redskins'].index)
nfl_teams["num_id"] = range(1,len(nfl_teams)+1)
#cleaning the nfl stadiums data
nfl_stadiums = nfl_stadiums[["stadium_name","stadium_location","stadium_open","stadium_close","stadium_type","stadium_capacity","stadium_surface","stadium_latitude","stadium_longitude"]]
nfl_stadiums["stadium_id"] = range(1,len(nfl_stadiums)+1)
#spread_longlat = spread.merge(nfl_stadiums[['stadium_longitude', 'stadium_latitude']], on='stadium_name', how='inner')
#heading the data to see
print(nfl_teams.head())
print(nfl_stadiums.head())
print(spread.head())
#spread_longlat.to_csv("spread_final.csv", index=False)

      schedule_date  season scheduled week  playoff          home team  \
13232      9/8/2022    2022              1    False   Los Angeles Rams   
13233     9/11/2022    2022              1    False  Arizona Cardinals   
13234     9/11/2022    2022              1    False    Atlanta Falcons   
13235     9/11/2022    2022              1    False  Carolina Panthers   
13236     9/11/2022    2022              1    False      Chicago Bears   

       home score  away score            away team favorite team  \
13232        10.0        31.0        Buffalo Bills           BUF   
13233        21.0        44.0   Kansas City Chiefs            KC   
13234        26.0        27.0   New Orleans Saints            NO   
13235        24.0        26.0     Cleveland Browns           CAR   
13236        19.0        10.0  San Francisco 49ers            SF   

       spread favor over under             stadium_name  id  
13232          -2.0         52             SoFi Stadium   1  
13233          -6.0   

now loading the data from a weather api to put into another dataframe

In [15]:
import requests
import pandas as pd 
from datetime import datetime

# Sample data loading
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file = os.path.join(data_dir, 'spreadwithlonglat.csv')
data = pd.read_csv(data_file,usecols=['schedule_date', 'stadium_latitude', 'stadium_longitude'])

data['schedule_date'] = pd.to_datetime(data['schedule_date'], format='%m/%d/%Y')
data['schedule_date'] = data['schedule_date'].dt.strftime('%Y-%m-%d')

# List of NFL game dates and locations.
games = data.to_dict(orient='records')

# Initialize a dictionary to store weather data for each game.
weather_data = {}

# Loop through the list of NFL games and make API requests.
for game in games:
    start = game['schedule_date']
    end = game['schedule_date']
    lat = game['stadium_latitude']
    lon = game['stadium_longitude']
    
    base_url = f'https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start}&end_date={end}&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,windspeed_10m_max&timezone=auto'
    # Make an API request to get historical weather data for the given date and location.
    r = requests.get(base_url)
    data = r.json()
    #print(data)
    weather_data[f'{start} - {lat},{lon}'] = data
df_weather=pd.DataFrame(data)


{'latitude': 42.100006, 'longitude': -71.299995, 'generationtime_ms': 0.09799003601074219, 'utc_offset_seconds': -14400, 'timezone': 'America/New_York', 'timezone_abbreviation': 'EDT', 'elevation': 84.0, 'daily_units': {'time': 'iso8601', 'temperature_2m_max': '°C', 'temperature_2m_min': '°C', 'temperature_2m_mean': '°C', 'windspeed_10m_max': 'km/h'}, 'daily': {'time': ['2017-09-07'], 'temperature_2m_max': [22.7], 'temperature_2m_min': [15.4], 'temperature_2m_mean': [18.6], 'windspeed_10m_max': [15.3]}}
{'latitude': 42.100006, 'longitude': -71.299995, 'generationtime_ms': 0.09799003601074219, 'utc_offset_seconds': -14400, 'timezone': 'America/New_York', 'timezone_abbreviation': 'EDT', 'elevation': 84.0, 'daily_units': {'time': 'iso8601', 'temperature_2m_max': '°C', 'temperature_2m_min': '°C', 'temperature_2m_mean': '°C', 'windspeed_10m_max': 'km/h'}, 'daily': {'time': ['2017-09-07'], 'temperature_2m_max': [22.7], 'temperature_2m_min': [15.4], 'temperature_2m_mean': [18.6], 'windspeed_1

KeyboardInterrupt: 

now loading the dataframes into sql

In [24]:
df_weather = pd.DataFrame.from_dict(weather_data, orient='columns')

In [25]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Uva!1819"
dst_dbname = "project1_db"
db_name = "project1_db"
pk_column="id"
df_weather["num_id"] = range(1,len(df_weather)+1)
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        #changing this to connection
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    connection.close()

db_operation = "insert"
tables = [('spread', spread, 'id'),
         ('teams',nfl_teams,'num_id'),
         ('stadium',nfl_stadiums,'stadium_id'),
         ('weather',df_weather,'num_id')]
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)